# Evaluation
This script demonstrates the basic evaluation of a trained model **on the validation set**. If your model works with this notebook unchanged, it should also work with the evaluation script we'll use for the test set.

Remember to use the `mamba` environment created from the `environment.yml` file provided with the code! Our evaluation script will use exactly that environment.

In [ ]:
%load_ext autoreload
%autoreload 2
# the autoreload magic is convenient when using a notebook while modifying the code in imported modules
# it also slows things down (because it re-imports the modules all the time), so you can just comment this out

import sys
sys.path.append("../")

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()
# Note: check with !nvidia-smi if there really are no available GPUs, can also be a PyTorch problem!
# try changing the line `- pytorch` to `- pytorch-gpu` in environment.yml when using an older GPU and/or older driver 

In [ ]:
import torch
import secrets
from src.utils import seed_torch

# Draw random seeds using Python's cryptographic library and cut them down to size
_global_seed = secrets.randbits(128) % (2**64-1)
_local_seed = secrets.randbits(128) % (2**64-1)

# If you want to reproduce a previous experiment (and only then) set the seeds to that previously recorded value
# and set `reproducible=True`. N.B.: using `42` is never a good idea!
seed_torch(_global_seed, reproducible=False)
_rng_cpu = torch.Generator()
_rng_cpu = _rng_cpu.manual_seed(_local_seed)

In [ ]:
from src.submission import get_group_number, get_group_names, get_my_model_name
print(f"Evaluating {get_group_number()} (members: {get_group_names()})\nwho have submitted their model as {get_my_model_name()}.pth")

In [ ]:
# If you need to change anything in this cell for this notebook to run, our evaluation script will also fail.

from src.submission import get_train_config, get_my_model, get_my_model_name, get_my_batch_tsfm
from src.datasets import get_preprocessed_ts_data
from src.utils import load_model
from pathlib import Path
from torch.utils.data import DataLoader
import torch.nn as nn

config = get_train_config()
config["targets"] = ["Spot", "wind_mean", "solar_mean"]
preprocess_kwargs = {
    "datetime_col": "Date",
    "data_interpolation_method": "linear",
    "valid_cutoff_datetime": "2025-01-01 00:00:00",
    "resolution_map": {
        'Spot': 'hour',
        'pumped hydro conso': 'hour',
        'cross-border trade': 'hour',
        'nuclear': 'hour',
        'run-of-the-river production': 'hour',
        'biomass': 'hour',
        'brown coal': 'hour',
        'black coal': 'hour',
        'oil': 'hour', 
        'natural gas': 'hour',
        'geothermal': 'hour',
        'stored hydro production': 'hour',
        'pumped hydro production': 'hour',
        'other': 'hour',
        'waste incineration': 'hour',
        'offshore wind': 'hour',
        'onshore wind': 'hour',
        'solar': 'hour',
        'conso': 'skip',
        'coal gas': 'hour',
        'CO2 price GER': 'week',
        'CO2 price EU': 'week',
        'biomass capacity': 'month',
        'offshore wind capacity': 'month',
        'onshore wind capacity': 'month',
        'solar capacity': 'month',
        'battery storage (nominal power)': 'month',
        'pumped hydro capacity': 'year',
        'brown coal capacity': 'year',
        'black coal capacity': 'year',
        'oil capacity': 'year',
        'natural gas capacity': 'year',
        'other (non-renewable) capacity': 'year',
        'Spot Gas EEX THE Day Ahead': 'week',
        'Spot coal': 'week',
        'wind_mean': 'hour',
        'solar_mean': 'hour',
    },
    "aggregation_methods_map": {
        'Spot': 'sum',
        'pumped hydro conso': 'sum',
        'cross-border trade': 'sum',
        'nuclear': 'sum',
        'run-of-the-river production': 'sum',
        'biomass': 'sum',
        'brown coal': 'sum',
        'black coal': 'sum',
        'oil': 'sum',
        'natural gas': 'sum',
        'geothermal': 'sum',
        'stored hydro production': 'sum',
        'pumped hydro production': 'sum',
        'other': 'sum',
        'waste incineration': 'sum',
        'offshore wind': 'sum',
        'onshore wind': 'sum',
        'solar': 'sum',
        'conso': 'mean',
        'coal gas': 'sum',
        'CO2 price GER': 'mean',
        'CO2 price EU': 'mean',
        'biomass capacity': 'mean',
        'offshore wind capacity': 'mean',
        'onshore wind capacity': 'mean',
        'solar capacity': 'mean',
        'battery storage (nominal power)': 'mean',
        'pumped hydro capacity': 'mean',
        'brown coal capacity': 'mean',
        'black coal capacity': 'mean',
        'oil capacity': 'mean',
        'natural gas capacity': 'mean',
        'other (non-renewable) capacity': 'mean',
        'Spot Gas EEX THE Day Ahead': 'mean',
        'Spot coal': 'mean',
        'wind_mean': 'mean',
        'solar_mean': 'mean',
    }
}

train_ds, valid_ds = get_preprocessed_ts_data(
    path="../data/ger_elec_price/energy_ger.parquet",
    features=config['features'],
    targets=config['targets'],
    window_size=config['forecast_history'],
    forecast_horizon=config['forecast_horizon'],
    preprocess_kwargs=preprocess_kwargs,
    stride=config["stride"]
)

train_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, pin_memory=torch.cuda.is_available(), generator=_rng_cpu)
val_loader = DataLoader(valid_ds, batch_size=config['batch_size'], shuffle=False, pin_memory=torch.cuda.is_available(), generator=_rng_cpu)

model = get_my_model(net_kwargs=config["net_params"], rng=_rng_cpu).to(device)
model.eval()
criterion = nn.MSELoss(reduction="mean")
batch_tsfm_fn = get_my_batch_tsfm()
save_model_path = Path(f"../models/{get_my_model_name()}.pth")


In [ ]:
# If you need to change anything in this cell for this notebook to run, our evaluation script will also fail.

print(f"Loading submitted model...\n")
load_model(model_path=save_model_path, model=model, optimizer=None, device=device, with_opt=False)
print(f"Submitted model architecture:\n{model}")

In [ ]:
from src.utils import predict_one_sample, plot_one_prediction

x, y = valid_ds[68]
pred, y = predict_one_sample(model, x, y, device, transform_batch_fn=get_my_batch_tsfm())
fig = plot_one_prediction(pred, y)
fig.show()

In [ ]:
x, y = valid_ds[44]
pred, y = predict_one_sample(model, x, y, device, transform_batch_fn=get_my_batch_tsfm())
fig = plot_one_prediction(pred, y)
fig.show()

In [ ]:
x, y = valid_ds[43]
pred, y = predict_one_sample(model, x, y, device, transform_batch_fn=get_my_batch_tsfm())
fig = plot_one_prediction(pred, y)
fig.show()

In [ ]:
x, y = valid_ds[42]
pred, y = predict_one_sample(model, x, y, device, transform_batch_fn=get_my_batch_tsfm())
fig = plot_one_prediction(pred, y)
fig.show()

In [ ]:
x, y = valid_ds[41]
pred, y = predict_one_sample(model, x, y, device, transform_batch_fn=get_my_batch_tsfm())
fig = plot_one_prediction(pred, y)
fig.show()